<p align="left"><img src="https://mospi.gov.in/sites/default/files/logo.png" width="500" height="100" /></p>

#  **Time-Use Survey & Consumer Pyramid**<br>
#### <p style= 'text-align:left'>Prepared by: ---- </p>                                          
---

### **Table of Contents**

1. [**Introduction**](#Section1)<br>

2. [**Data Acquisition & Description**](#Section4)<br>

3. [**Exploratory Data Analysis**](#Section6)<br>

4. [**Summarization**](#Section7)<br>


<a name = Section1></a>
### **1. Introduction**
---
--


<a name = Section4></a>
### **2. Data Acquisition & Description**

##### This section is emphasised on the accquiring the data, cleaning it and obtain some descriptive information out of it.
---



In [28]:
# #Installing/Upgrading Libraries

# !pip install pandas
# !pip install dask pandas
# !pip install scipy

# !pip install -q datascience                                         # Package that is required by pandas profiling
# !pip install ydata-profiling

# !pip install tabula-py
# !pip install nbconvert
# !pip install geopy

In [7]:
# Importing Libraries
# -------------------------------------------------------------------------------------------------------------------------------
# Importing library for scientific calculations
import scipy.stats as st
# Importing package pandas (For Panel Data Analysis)
import pandas as pd
# import dask.dataframe as dd
# Import Pandas Profiling (To generate Univariate Analysis)
# from ydata_profiling import ProfileReport
# -------------------------------------------------------------------------------------------------------------------------------
# Importing package numpys (For Numerical Python)
import numpy as np
# -------------------------------------------------------------------------------------------------------------------------------
# Importing pyplot interface to use matplotlib
import matplotlib.pyplot as plt
# Importing seaborn library for interactive visualization
import seaborn as sns
# -------------------------------------------------------------------------------------------------------------------------------
#Importing Tabula to read tables from pdf
# from tabula import read_pdf
# -------------------------------------------------------------------------------------------------------------------------------
from datetime import datetime, timedelta
# -------------------------------------------------------------------------------------------------------------------------------
import nbconvert
# -------------------------------------------------------------------------------------------------------------------------------
import warnings                                                     # Importing warning to disable runtime warnings
warnings.filterwarnings("ignore")                                   # Warnings will appear only once
# -------------------------------------------------------------------------------------------------------------------------------
import os
import glob

# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="MyApp")

##### **Fetching & Cleaning Consumer Pyramid**:

In [6]:
cp_dummy = pd.read_stata(filepath_or_buffer= '/Users/gulatii16/Library/CloudStorage/GoogleDrive-himanshu.gulatii16@gmail.com/My Drive/0 g [n]/1 Focus Areas/3 Data Science/CROPPED VARIABLES_MERGED.dta',iterator= False)
pd.set_option('display.max_columns',10)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/gulatii16/Library/CloudStorage/GoogleDrive-himanshu.gulatii16@gmail.com/My Drive/0 g [n]/1 Focus Areas/3 Data Science/CROPPED VARIABLES_MERGED.dta'

In [99]:
sorted_cp_dummy = cp_dummy.sort_values(by=['PID','TIME'])
sorted_cp_dummy['TIME'] = sorted_cp_dummy['TIME'] -1 

sorted_cp_dummy = sorted_cp_dummy[(sorted_cp_dummy['relation_with_hoh'] == 'HOH') | (sorted_cp_dummy['relation_with_hoh'] == 'Spouse') | (sorted_cp_dummy['relation_with_hoh'] == 'Son') | (sorted_cp_dummy['relation_with_hoh'] == 'Daughter') | (sorted_cp_dummy['relation_with_hoh'] == 'Son-in-law') |  (sorted_cp_dummy['relation_with_hoh'] == 'Daughter-in-law')]

code_mapping = {'Widowed': 8, 'Married': 5, 'Unmarried': 7,'Not Applicable': 6,'Divorced/Separated': 4}
sorted_cp_dummy['marital_status_code'] = sorted_cp_dummy['marital_status'].map(code_mapping)

sorted_cp_dummy['lag_marital_status_code'] = sorted_cp_dummy.groupby(by=['PID'])[['marital_status_code']].shift(1)
sorted_cp_dummy['lead_marital_status_code'] = sorted_cp_dummy.groupby(by=['PID'])[['marital_status_code']].shift(-1)

sorted_cp_dummy['married_to_widow'] = np.where((sorted_cp_dummy['lag_marital_status_code'] == 5) & (sorted_cp_dummy['marital_status_code'] == 8), True, False)
sorted_cp_dummy['married_to_widow_filtered'] = np.where(((sorted_cp_dummy['lag_marital_status_code'] == 5) & (sorted_cp_dummy['marital_status_code'] == 8)) & ((sorted_cp_dummy['relation_with_hoh'] == 'HOH') | (sorted_cp_dummy['relation_with_hoh'] == 'Spouse')) & ((sorted_cp_dummy['age_yrs'] >= 12) & (sorted_cp_dummy['age_yrs'] <= 75)), True, False)

# sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['married_to_widow_filtered'].value_counts()
# sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['PID'].nunique()

#13803/14437
#11702/12284

# Basically, few PIDs have multiple maried to widow status true, which seems to be an outlier & needs to be removed

PID_married_to_widow = sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['PID'].value_counts().to_frame().reset_index()
PID_married_to_widow = PID_married_to_widow[PID_married_to_widow['count']>1]
PID_to_remove_list = PID_married_to_widow['PID'].to_list()
# len(PID_to_remove_list)

# 1828/1882 PIDs need to be removed because they reported multiple change in married to widow status

sorted_cp_dummy = sorted_cp_dummy[~sorted_cp_dummy['PID'].isin(PID_to_remove_list)]

# sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['PID'].nunique()
# sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['hh_id'].nunique()

# 9874/10402 shocks (unique PIDs with status change only once) happened in 9848/10360 HHIDs

# sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['hh_id'].value_counts().to_frame()['count'].value_counts()

# count
# 1    9822/10318
# 2      26/42

# 2 deaths happen in 26/42 HHIDs out of 9948/10360 HHIDs

shock_hhids = sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['hh_id'].value_counts().to_frame().reset_index()
sorted_cp_dummy['whether_shock_hhid'] = np.where(sorted_cp_dummy['hh_id'].isin(shock_hhids['hh_id']), True, False)

# sorted_cp_dummy.loc[sorted_cp_dummy['whether_shock_hhid'] == True]['PID'].nunique()
# 9874/10402 shocks happened in 9848/10360 HHIDs affected 35884/33030 individuals (226316/228458 observations, since not all PIDs have all 11 waves)

shock_pids = sorted_cp_dummy.loc[sorted_cp_dummy['married_to_widow_filtered'] == True]['PID'].value_counts().to_frame().reset_index()
sorted_cp_dummy['whether_shock_PIDs'] = np.where(sorted_cp_dummy['PID'].isin(shock_pids['PID']), True, False)

# sorted_cp_dummy.loc[sorted_cp_dummy['whether_shock_PIDs'] == True]['PID'].count()
# 9874/10402 shocks meaning 9874/10402 people died which have a total of 79344/80022 observations (pre & post shock)

sorted_cp_dummy['shock_wave_num_statuschangePID'] = np.where(sorted_cp_dummy['married_to_widow_filtered'] == True,sorted_cp_dummy['TIME'],0)
# sorted_cp_dummy['shock_wave_num_statuschangePID'].value_counts()
#checked: value copied to 9874/10402 PIDs whose satus got changed

representative_values = sorted_cp_dummy.groupby('hh_id')['shock_wave_num_statuschangePID'].max()
sorted_cp_dummy['shock_wave_num_allAffectedPID'] = np.where(sorted_cp_dummy['whether_shock_hhid'] == True, sorted_cp_dummy['hh_id'].map(representative_values),0)
# sorted_cp_dummy['shock_wave_num_allAffectedPID'].value_counts()
#checked: value copied to 226316/228458 observations corresponding to 35884/80022 PIDs of 9848/10360 HHIDs

sorted_cp_dummy['current_time-shock_time'] = np.where(sorted_cp_dummy['whether_shock_hhid'] == True, sorted_cp_dummy['TIME']-sorted_cp_dummy['shock_wave_num_allAffectedPID'],np.NaN)
sorted_cp_dummy['pre_to_post_shock_status'] = np.where(sorted_cp_dummy['current_time-shock_time']>=0,1,np.where(sorted_cp_dummy['current_time-shock_time']<0,0,np.NaN))

# sorted_cp_dummy['pre_to_post_shock_status'].value_counts()
# Out of 226316/228458 observations of 35884/33030 PIDs in 9848/10360 hhids, 99012/100796 observations are after shock observations

# sorted_cp_dummy.loc[sorted_cp_dummy['whether_shock_PIDs']==True].groupby(by=['PID'])[['gender']].nunique()[['gender']].value_counts()
# gender
# 1         10380
# 2            22
# 22 directly affected people have 2 genders reported in different waves, hence, hhids of these people need to be removed

df = sorted_cp_dummy.loc[sorted_cp_dummy['whether_shock_PIDs']==True].groupby(by=['PID'])[['gender']].nunique()[['gender']]
df = df[df['gender']>1]
PID_directlyAffected_to_remove_list = df[['gender']].index.to_list()
hhid_directlyAffected_to_remove_list = sorted_cp_dummy[sorted_cp_dummy['PID'].isin(PID_directlyAffected_to_remove_list)]['hh_id'].to_list()

sorted_cp_dummy = sorted_cp_dummy[~sorted_cp_dummy['hh_id'].isin(hhid_directlyAffected_to_remove_list)]

# sorted_cp_dummy['PID'].nunique()
# sorted_cp_dummy.groupby(by=['PID'])[['gender']].nunique()[['gender']].value_counts()
# 506129 unique people are there, out of which 1056 have 2 genders reported in different waves, hence being removed.

df = sorted_cp_dummy.groupby(by=['PID'])[['gender']].nunique()[['gender']]
df = df[df['gender']>1]
PID_ALL_to_remove_list = df[['gender']].index.to_list()
hhid_ALL_to_remove_list = sorted_cp_dummy[sorted_cp_dummy['PID'].isin(PID_ALL_to_remove_list)]['hh_id'].to_list()

# sorted_cp_dummy[(sorted_cp_dummy['PID'].isin(PID_ALL_to_remove_list)) & (sorted_cp_dummy['whether_shock_hhid'] == True)]['PID'].nunique()

sorted_cp_dummy = sorted_cp_dummy[~sorted_cp_dummy['hh_id'].isin(hhid_ALL_to_remove_list)]

# sorted_cp_dummy.loc[sorted_cp_dummy['whether_shock_hhid'] == True]['PID'].nunique()

# Unique PIDs (with shock) have been reduced from 33030 to 32666 which is 364
# Out of these 149 PIDs (with shock) removed, 22 are directly affected & 342 are indirectly affected
# Out of these 342 indirectly affected, 287 don't belong to hhids where gender of directly affected person changes
# Hence, 22+55 people removed from hhids where directly affected person has gender changed & 287 elsewhere
# In total, 364 PIDs are removed

In [120]:
religion_mode_frame = sorted_cp_dummy.groupby(by = ['PID'])['religion'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else None).to_frame()
religion_mode_frame = religion_mode_frame.reset_index()
religion_mode_frame.rename(columns={'religion':'new_religion'},inplace = True)
sorted_cp_dummy =  pd.merge(sorted_cp_dummy, religion_mode_frame[['PID', 'new_religion']], on='PID', how='left')

religion_mode_frame = sorted_cp_dummy.groupby(by = ['PID'])['caste'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else None).to_frame()
religion_mode_frame = religion_mode_frame.reset_index()
religion_mode_frame.rename(columns={'caste':'new_caste'},inplace = True)
sorted_cp_dummy =  pd.merge(sorted_cp_dummy, religion_mode_frame[['PID', 'new_caste']], on='PID', how='left')

religion_mode_frame = sorted_cp_dummy.groupby(by = ['PID'])['caste_category'].agg(lambda x: x.mode().iat[0] if not x.mode().empty else None).to_frame()
religion_mode_frame = religion_mode_frame.reset_index()
religion_mode_frame.rename(columns={'caste_category':'new_caste_category'},inplace = True)
sorted_cp_dummy =  pd.merge(sorted_cp_dummy, religion_mode_frame[['PID', 'new_caste_category']], on='PID', how='left')

In [121]:
sorted_cp_dummy['daughter_son'] = np.where(sorted_cp_dummy['relation_with_hoh']=='Daughter',1,np.where(sorted_cp_dummy['relation_with_hoh']=='Son',0,np.NaN))

In [122]:
# setting negative ts to zero
sorted_cp_dummy['ts_on_indoor_entmt'] = np.where(sorted_cp_dummy['ts_on_indoor_entmt']<0,np.NaN,sorted_cp_dummy['ts_on_indoor_entmt'])
sorted_cp_dummy['ts_on_learning'] = np.where(sorted_cp_dummy['ts_on_learning']<0,np.NaN,sorted_cp_dummy['ts_on_learning'])
sorted_cp_dummy['ts_on_oth_activities_for_self'] = np.where(sorted_cp_dummy['ts_on_oth_activities_for_self']<0,np.NaN,sorted_cp_dummy['ts_on_oth_activities_for_self'])
sorted_cp_dummy['ts_on_outdoor_sports'] = np.where(sorted_cp_dummy['ts_on_outdoor_sports']<0,np.NaN,sorted_cp_dummy['ts_on_outdoor_sports'])
sorted_cp_dummy['ts_on_religious_activities'] = np.where(sorted_cp_dummy['ts_on_religious_activities']<0,np.NaN,sorted_cp_dummy['ts_on_religious_activities'])
sorted_cp_dummy['ts_on_travel'] = np.where(sorted_cp_dummy['ts_on_travel']<0,np.NaN,sorted_cp_dummy['ts_on_travel'])
sorted_cp_dummy['ts_on_work_as_unpaid_trainee'] = np.where(sorted_cp_dummy['ts_on_work_as_unpaid_trainee']<0,np.NaN,sorted_cp_dummy['ts_on_work_as_unpaid_trainee'])
sorted_cp_dummy['ts_on_work_as_unpaid_volunteer'] = np.where(sorted_cp_dummy['ts_on_work_as_unpaid_volunteer']<0,np.NaN,sorted_cp_dummy['ts_on_work_as_unpaid_volunteer'])
sorted_cp_dummy['ts_on_work_for_employer'] = np.where(sorted_cp_dummy['ts_on_work_for_employer']<0,np.NaN,sorted_cp_dummy['ts_on_work_for_employer'])
sorted_cp_dummy['ts_on_work_for_hh_n_mem'] = np.where(sorted_cp_dummy['ts_on_work_for_hh_n_mem']<0,np.NaN,sorted_cp_dummy['ts_on_work_for_hh_n_mem'])
sorted_cp_dummy['ts_with_frnds'] = np.where(sorted_cp_dummy['ts_with_frnds']<0,np.NaN,sorted_cp_dummy['ts_with_frnds'])

# sorted_cp_dummy['ts_on_indoor_entmt'].value_counts().sum()

# setting all zeros to np.NaN
sorted_cp_dummy['ts_on_indoor_entmt_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_indoor_entmt']==0,np.NaN,sorted_cp_dummy['ts_on_indoor_entmt'])
sorted_cp_dummy['ts_on_learning_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_learning']==0,np.NaN,sorted_cp_dummy['ts_on_learning'])
sorted_cp_dummy['ts_on_oth_activities_for_self_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_oth_activities_for_self']==0,np.NaN,sorted_cp_dummy['ts_on_oth_activities_for_self'])
sorted_cp_dummy['ts_on_outdoor_sports_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_outdoor_sports']==0,np.NaN,sorted_cp_dummy['ts_on_outdoor_sports'])
sorted_cp_dummy['ts_on_religious_activities_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_religious_activities']==0,np.NaN,sorted_cp_dummy['ts_on_religious_activities'])
sorted_cp_dummy['ts_on_travel_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_travel']==0,np.NaN,sorted_cp_dummy['ts_on_travel'])
sorted_cp_dummy['ts_on_work_as_unpaid_trainee_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_work_as_unpaid_trainee']==0,np.NaN,sorted_cp_dummy['ts_on_work_as_unpaid_trainee'])
sorted_cp_dummy['ts_on_work_as_unpaid_volunteer_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_work_as_unpaid_volunteer']==0,np.NaN,sorted_cp_dummy['ts_on_work_as_unpaid_volunteer'])
sorted_cp_dummy['ts_on_work_for_employer_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_work_for_employer']==0,np.NaN,sorted_cp_dummy['ts_on_work_for_employer'])
sorted_cp_dummy['ts_on_work_for_hh_n_mem_only_ptcpt'] = np.where(sorted_cp_dummy['ts_on_work_for_hh_n_mem']==0,np.NaN,sorted_cp_dummy['ts_on_work_for_hh_n_mem'])
sorted_cp_dummy['ts_with_frnds_only_ptcpt'] = np.where(sorted_cp_dummy['ts_with_frnds']==0,np.NaN,sorted_cp_dummy['ts_with_frnds'])

# sorted_cp_dummy['ts_on_indoor_entmt_only_ptcpt'].value_counts().sum()

# setting all np.NaN to zeros
sorted_cp_dummy['ts_on_indoor_entmt_all_NAN_zeros'] = sorted_cp_dummy['ts_on_indoor_entmt'].fillna(0)
sorted_cp_dummy['ts_on_learning_all_NAN_zeros'] = sorted_cp_dummy['ts_on_learning'].fillna(0)
sorted_cp_dummy['ts_on_oth_activities_for_self_all_NAN_zeros'] = sorted_cp_dummy['ts_on_oth_activities_for_self'].fillna(0)
sorted_cp_dummy['ts_on_outdoor_sports_all_NAN_zeros'] = sorted_cp_dummy['ts_on_outdoor_sports'].fillna(0)
sorted_cp_dummy['ts_on_religious_activities_all_NAN_zeros'] = sorted_cp_dummy['ts_on_religious_activities'].fillna(0)
sorted_cp_dummy['ts_on_travel_all_NAN_zeros'] = sorted_cp_dummy['ts_on_travel'].fillna(0)
sorted_cp_dummy['ts_on_work_as_unpaid_trainee_all_NAN_zeros'] = sorted_cp_dummy['ts_on_work_as_unpaid_trainee'].fillna(0)
sorted_cp_dummy['ts_on_work_as_unpaid_volunteer_all_NAN_zeros'] = sorted_cp_dummy['ts_on_work_as_unpaid_volunteer'].fillna(0)
sorted_cp_dummy['ts_on_work_for_employer_all_NAN_zeros'] = sorted_cp_dummy['ts_on_work_for_employer'].fillna(0)
sorted_cp_dummy['ts_on_work_for_hh_n_mem_all_NAN_zeros'] = sorted_cp_dummy['ts_on_work_for_hh_n_mem'].fillna(0)
sorted_cp_dummy['ts_with_frnds_all_NAN_zeros'] = sorted_cp_dummy['ts_with_frnds'].fillna(0)

# sorted_cp_dummy['ts_on_indoor_entmt_all_NAN_zeros'].value_counts().sum()

sorted_cp_dummy['total_ts'] = sorted_cp_dummy['ts_on_indoor_entmt_all_NAN_zeros'] + sorted_cp_dummy['ts_on_learning_all_NAN_zeros'] + sorted_cp_dummy['ts_on_oth_activities_for_self_all_NAN_zeros'] + sorted_cp_dummy['ts_on_outdoor_sports_all_NAN_zeros'] + sorted_cp_dummy['ts_on_religious_activities_all_NAN_zeros'] + sorted_cp_dummy['ts_on_travel_all_NAN_zeros'] + sorted_cp_dummy['ts_on_work_as_unpaid_trainee_all_NAN_zeros'] + sorted_cp_dummy['ts_on_work_as_unpaid_volunteer_all_NAN_zeros'] + sorted_cp_dummy['ts_on_work_for_employer_all_NAN_zeros'] + sorted_cp_dummy['ts_on_work_for_hh_n_mem_all_NAN_zeros'] + sorted_cp_dummy['ts_with_frnds_all_NAN_zeros']
sorted_cp_dummy['total_ts'].value_counts()

# total_ts
# 24.0    3543163
# 0.0      181354
# 16.0         51
# 17.0         11
# 18.0          7
# 15.0          6
# 15.5          4
# 21.0          3
# 20.5          3
# 20.0          3
# 16.5          2
# 19.0          2
# 14.0          2
# 17.5          2
# 18.5          1
# Name: count, dtype: int64

total_ts
24.0    3473019
0.0      173755
16.0         48
17.0         10
18.0          7
15.0          6
15.5          4
21.0          3
20.5          3
20.0          3
16.5          2
19.0          2
14.0          2
18.5          1
17.5          1
Name: count, dtype: int64

In [123]:
religion_mode_frame['new_caste_category'].value_counts()

new_caste_category
OBC                   197382
Upper Caste           118287
SC                    106516
Intermediate Caste     46160
ST                     28793
Not Stated              4735
Not Applicable            35
Name: count, dtype: int64

In [124]:
sorted_cp_dummy.to_stata('sorted_cp_dummy.dta')

: 

In [ ]:
affected_cp_dummy = sorted_cp_dummy[sorted_cp_dummy['whether_shock_hhid'] == True]
affected_cp_dummy.to_stata('affected_cp_dummy.dta')

ValueError: Column `new_religion` cannot be exported.

Only string-like object arrays
containing all strings or a mix of strings and None can be exported.
Object arrays containing only null values are prohibited. Other object
types cannot be exported and must first be converted to one of the
supported types.

In [28]:
# affected_cp_dummy = pd.read_stata(filepath_or_buffer = '/Users/gulatii16/Library/CloudStorage/GoogleDrive-himanshu.gulatii16@gmail.com/My Drive/0 g [n]/5 Part Time Works/6 Shila Work/5 Exogenous Shock Study_CMIE/affected_cp_dummy_04012024.dta')


<a name = Section6></a>
### **3. Exploratory Data Analysis**
##### This section is emphasised on asking the right questions and perform analysis using the data.
---

---
<a name = Section7></a>
### **4. Summarization: Conclusion & Actionable Insights**
---

#### **Saving the data into html:**

In [ ]:
# !jupyter nbconvert Commutation_Analysis.ipynb --no-input --to html